In [5]:
import requests
import pandas as pd
from tqdm import tqdm
import time

In [6]:
# Read the csv that I had with just the titles of Animes
anime_list = pd.read_csv("../Data/Anime.csv", sep = ";")
anime_list.head(10)

,Title,Q_views
0,Dragon Ball,1
1,Pokemon,1
2,Sailor Moon,1
3,Inuyasha,7
4,Death note,2
5,Kakegurui,1
6,Hunter x hunter (2011),2
7,Hunter x Hunter Movie 2: The Last Mission,1
8,Hunter x Hunter Movie 1: Phantom Rouge,1
9,shingeki no kyoyin,2


In [7]:
# API's configuration
BASE_URL = "https://api.jikan.moe/v4"

def get_anime_data(title):
    try:
        response = requests.get(f"{BASE_URL}/anime?q={title}&limit=1")
        if response.status_code == 200:
            data = response.json()
            if data.get("data"):
                anime = data["data"][0]
                return {
                    "title": anime["title"],
                    "genres": [g["name"] for g in anime["genres"]],
                    "episodes": anime["episodes"],
                    "duration_per_episode": anime["duration"],  # Formato: "24 min"
                    "score": anime["score"],
                    "year": anime["year"],
                    "studios": [g["name"] for g in anime["studios"]]
                }
        return None
    except Exception as e:
        print(f"Error fetching {title}: {e}")
        return None


In [8]:
# Scraping for each anime
enriched_data = []
for idx, title in enumerate(tqdm(anime_list["Title"])):
    anime_data = get_anime_data(title)
    if anime_data:
        anime_data["Q_views"] = anime_list.loc[idx, "Q_views"]
        enriched_data.append(anime_data)
    time.sleep(1)  # Evitar rate limits

100%|███████████████████████████████████████████| 58/58 [01:27<00:00,  1.52s/it]


In [9]:
# Create new dataframe
df_enriched = pd.DataFrame(enriched_data)

In [10]:
df_enriched.head()

,title,genres,episodes,duration_per_episode,score,year,studios,Q_views
0,Dragon Ball,"[Action, Adventure, Comedy, Fantasy]",153,24 min per ep,7.98,1986.0,[Toei Animation],1
1,Pokemon,"[Action, Adventure, Comedy, Fantasy]",276,24 min per ep,7.41,1997.0,[OLM],1
2,Bishoujo Senshi Sailor Moon,[Romance],46,24 min per ep,7.74,1992.0,[Toei Animation],1
3,InuYasha,"[Action, Adventure, Fantasy, Romance]",167,24 min per ep,7.87,2000.0,[Sunrise],7
4,Death Note,"[Supernatural, Suspense]",37,23 min per ep,8.62,2006.0,[Madhouse],2


In [11]:
# Save the dataframe
df_enriched.to_csv("../Data/anime_data.csv", index=False)